# Yelp API - Lab



## Introduction 

Now that we've seen how the Yelp API works, it's time to put those API and SQL skills to work in order to do some basic business analysis! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to:
* Create a DB on AWS to store information from Yelp about businesses
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and insert the information into your DB
* Perform pagination to retrieve troves of data!
* Write SQL queries to answer questions about your data 

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner and combine it with our other data skills. In this lab you will get data from the Yelp API, store that data in a SQL Database on AWS, and write queries to answer follow-up questions. 


### Outline:

1. Determine which pieces of information you need to pull from the Yelp API.

2. Create a DB schema with 2 tables. One for the businesses and one for the reviews.

3. Create Python functions to:
  - Perform a search of businesses using pagination
  - Parse the API response for specific data points
  - Insert the data into your AWS DB

4. Use the functions above in a loop that will paginate over the results to retrieve all of the results. 

*Something might cause your code to break while it is running. You don't want to constantly repull the same data when this happens, so you should insert the data into the database as you call and parse it, not after you have all of the data*

5. Create functions to:
  - Retrieve the reviews data of one business
  - Parse the reviews response for specific review data
  - Insert the review data into the DB

6. Using SQL, query all of the business IDs. Using the 3 Python functions you've created, run your business IDs through a loop to get the reviews for each business and insert them into your DB.

7. Write SQL queries to answer the following questions about your data.


Bonus Steps:  
- Place your helper functions in a package so that your final notebook only has the major steps listed.
- Rewrite your business search functions to be able take an argument for the type of business you are searching for.
- Add another group of businesses to your database.


 
## SQL Questions:

- What are the 5 businesses with the highest average ratings?
- What are the 5 businesses with the lowest average ratings?
- What is the average rating of restaurants that have a price label of one dollar sign? Two dollar signs? Three dollar signs? 
- How many businesses have a rating greater than or  4.5?
- How many businesses have a rating less than 3?
- Return the text of the reviews for the most reviewed restaurant. 
- Find the highest rated business and return text of the most recent review. If multiple business have the same rating, select the restaurant with the most reviews. 
- Find the lowest rated business and return text of the most recent review.  If multiple business have the same rating, select the restaurant with the least reviews. 


In [86]:
import mysql.connector 
import config
import pandas as pd
from mysql.connector import errorcode
import json
import requests

## Part I - Set up the DB

Start by reading SQL questions above to get an understanding of the data you will need. Then, read the documentation of Yelp API to understand what data you will receive in the response.  


Now that you are familiar with the data, create your SQL queries to create the DB and the appropriate tables. 

In [87]:
## Connect to DB server on AWS
cnx = mysql.connector .connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = 'yelp_data'
)

# print(cnx)



In [88]:
#select the cursor 
cursor = cnx.cursor()

In [6]:
## Create new DB
# cursor.execute("CREATE DATABASE yelp_data")


In [5]:
#in case I need to drop the database
# cursor.execute("""DROP DATABASE yelp_data""")

In [9]:
# Create a table for the Businesses
TABLES = {}
TABLES['businesses'] =  ("""CREATE TABLE IF NOT EXISTS businesses 
                        (id varchar (50) not null unique, 
                        name varchar (50) not null, 
                        review_count int(30) not null, 
                        rating float(20) not null,
                        price float (20) not null,
                        PRIMARY KEY (id))
                        engine=innoDB ;
                        """)


In [10]:
# TABLES['businesses']
cursor.execute(TABLES['businesses'])


In [89]:
cursor = cnx.cursor()
cursor.close()
cnx.close()

In [94]:
cnx = mysql.connector .connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = 'yelp_data'
)

cursor = cnx.cursor()

In [97]:
# Create a table for the reviews  ---  Come back to this!!!

TABLES['reviews'] = ("""CREATE TABLE reviews
                    (r_id varchar (50) not null,
                    review_text longblob not null,
                    time_created datetime not null,
                    PRIMARY KEY(r_id),
                    FOREIGN KEY (id) REFERENCES businesses.id))
                    ENGINE=innoDB;
                    """)

In [98]:
cursor.execute(TABLES['reviews'])

ProgrammingError: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near ')
                    ENGINE=innoDB' at line 6

In [41]:
# cursor.execute("""DROP TABLE Reviews""")

## Part 2: Create ETL pipeline for the business data from the API

In [61]:
# write a function to make a call to the DB
def indian_restaurants_bk(SEARCH_LIMIT):
    term = 'Indian'
    location = 'Brooklen NY'
    SEARCH_LIMIT = 10

    url = 'https://api.yelp.com/v3/businesses/search'

    headers = {
            'Authorization': 'Bearer {}'.format(config.yelpkey),
        }

    url_params = {
                    'term': term.replace(' ', '+'),
                    'location': location.replace(' ', '+'),
                    'limit': SEARCH_LIMIT
                }
    response = requests.get(url, headers=headers, params=url_params)

    results = json.loads(response.content)
    #print(results)
    
    
    return results
#     print(response)
    #print(type(response.text))
# print(response.text[:1000])




In [76]:
#call the restaurants function
data = indian_restaurants_bk(10)

In [85]:
type(data['businesses'])

list

# Parsing Business Data 

In [77]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = 'yelp_data'
)
cursor = cnx.cursor()

In [78]:
business_columns = ['id', 'name', 'review_count', 'rating', 'price']

In [79]:
# write a function to parse the API response 
# so that you can easily insert the data in to the DB
def parse_business_results(api_call):
    df = pd.DataFrame(api_call['businesses'])
    df = df[business_columns]
    df['price'] = df['price'].map(lambda x: len(x),na_action='ignore')
    return df

#na action specifies what happens when there is a null value
parse_business_results(data)

,id,name,review_count,rating,price
0,ZqOxAKVZEZTa4XS4zthp3Q,Bombay Grill,199,4.5,2
1,sCC7-hSdCkNPExejZT9BAQ,The MasalaWala,1426,4.5,2
2,dGE1Imqoz1vdHa2cjfvO1w,Gandhi Fine Indian Cuisine,358,4.0,2
3,ik7LcD1fkzVbVQ6Sntd4LA,Indian Spice,155,4.0,2
4,BXoW0AUDE7WZJcY-Vhqlbg,Indian Table,99,4.5,2
5,9eZrG8nANKuoyGSqdj_gew,Ashoka Grill,183,4.0,2
6,lxNU-aBq0yPwBDx-i89ahA,Seva Indian Cuisine,1173,4.5,2
7,x0k5kFArHLijdY8-NEQ7Xg,Bengal Tiger Indian Food,1757,4.5,2
8,hevbe9F6sYPguKO5whPSlA,India's Curry,103,4.0,2
9,EKY8DwZVWpkgv6IWrepbWg,Namaste,182,4.0,2


## Part 3: Create ETL pipeline for the restaurant review data from the API

In [99]:
# write a query to pull back all of the business ids 
# you will need these ids to pull back the reviews for each restaurant
def yelp_api_call_reviews(r_ids):
    outputs = []
    for r_id in r_ids:
        url = 'https://api.yelp.com/v3/businesses/{}/reviews'.format(r_id)
        headers = {'Authorization': 'Bearer {}'.format(config.api_key)}

        response = requests.get(url, headers=headers)
    
        outputs.append(response.content)
        
        outputs = [json.loads(x) for x in outputs]
        
        dfs = []

        for item in outputs:
            dfs.append(pd.DataFrame(item['reviews']))
            df = pd.concat(dfs)
            df = df[['id','text','time_created']]
            df['business_id'] = r_id
            #return [tuple(x) for x in df.values]

            return df.head()


In [ ]:
# write a function that takes a business id 
# and makes a call to the API for reivews
# then parse out the relevant information

In [ ]:
# write a function to insert the parsed data into the reviews table

## Part 4: Write SQL queries that will answer the questions posed. 

###  Pagination

Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. 

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and APIs can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

***Below is sample code that you can use to help you deal with the pagination parameter.***

In [ ]:
# Your code here; use a function or loop to retrieve all the results from your original request
import time

def yelp_call(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    
    data = response.json()['businesses']
    return data

def all_results(url_params, api_key):
    num = response.json()['total']
    print('{} total matches found.'.format(num))
    cur = 0
    results = []
    
    while cur < num and cur < 1000:
        url_params['offset'] = cur
        results.append(yelp_call(url_params, api_key))
        time.sleep(1) #Wait a second
        results += 50
    return df

term = 'pizza'
location = 'Astoria NY'
url_params = {  'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit' : 50
             }
df = all_results(url_params, api_key)
print(len(df))
df.head()

### Sample SQL Query 

Below is a SQL query to create a table.  Additionally here is a link to create a table with a foreign key.

http://www.mysqltutorial.org/mysql-foreign-key/

```CREATE TABLE IF NOT EXISTS tasks (
    task_id INT AUTO_INCREMENT,
    title VARCHAR(255) NOT NULL,
    start_date DATE,
    due_date DATE,
    status TINYINT NOT NULL,
    priority TINYINT NOT NULL,
    description TEXT,
    PRIMARY KEY (task_id)
)  ENGINE=INNODB;```